### Задание

1. Обучить модель на понравившихся данных
2. Создать rest api сервис, к которому можно будет обращаться для получения прогнозов

### Примерные темы

1. Определение токсичности комментария
2. Определение стоимости жилья от разных факторов: площадь жилья, удаленность от центра, etc (здесь же куча идей прогнозирования стоимости чего-либо по его описанию)
3. Темы новости по ее тексту
4. Классификатор рукописных цифр
5. Классификатор картинок (например, определение города и страны по снимку улицы)
6. Определение вероятности наличия сердечно-сосудистых заболеваний по данным первичного осмотра (или что-то подобное)
7. Многое другое (придумайте сами)
8. Прогнозирование рейтинга вопроса на stackoverflow - https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate
9. https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016
10. Тематическое моделирование статей на arxiv - https://www.kaggle.com/Cornell-University/arxiv

### Откуда брать данные

1. kaggle (https://www.kaggle.com/datasets)
2. спарсить самостоятельно, но в этом случае вам может понадобиться разметка (если у вас обучение с учителем)

Пишем простой граббер для Telegram чатов на Python https://medium.com/swlh/build-a-telegram-bot-in-go-in-9-minutes-e06ad38acef1  https://proglib.io/p/pishem-prostoy-grabber-dlya-telegram-chatov-na-python-2019-11-06

Spam Classifier in Python from scratch https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73

In [1]:
import pandas as pd
import numpy as np
import re

#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
                                precision_recall_curve, confusion_matrix
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def clean_msg(msg):
    if isinstance(msg, list):
        list_len = len(msg) - 2
        for pos, elem in enumerate(msg):
            if isinstance(elem, dict) and pos ==  list_len:
                for key, val in elem.items():
                    if key == 'type' and val == 'hashtag':
                        break
                    if key == 'type' and val == 'bold':
                        break
                    if key == 'type' and val == 'mention_name':
                        break
                    if key == 'text':
                        return val
    if isinstance(msg, str):
        return msg


In [3]:
ham_raw = pd.read_json('../../data/spam_detection/ham2016.json')
spam_raw = pd.read_json('../../data/spam_detection/spam.json')

## Обработка неспамных сообщений

In [4]:
ham_raw.head(3)

,name,type,id,messages
0,Новое Измайлово,public_supergroup,9666949336,"{'id': -999996617, 'type': 'service', 'date': ..."
1,Новое Измайлово,public_supergroup,9666949336,"{'id': 1, 'type': 'service', 'date': '2016-08-..."
2,Новое Измайлово,public_supergroup,9666949336,"{'id': 99, 'type': 'service', 'date': '2016-09..."


In [5]:
ham_raw1 = pd.json_normalize(ham_raw['messages'])
pd.set_option('display.max_columns', 50)
ham_raw1.tail(3)

,id,type,date,actor,actor_id,action,text,title,members,from,from_id,file,thumbnail,media_type,sticker_emoji,width,height,reply_to_message_id,photo,edited,forwarded_from,mime_type,contact_information.first_name,contact_information.last_name,contact_information.phone_number,duration_seconds,via_bot,contact_vcard,inviter,message_id,poll.question,poll.closed,poll.total_voters,poll.answers,location_information.latitude,location_information.longitude
60987,73759,message,2021-03-24T09:52:51,NaN,NaN,NaN,мойка проходит в несколько этапов: сначала сби...,NaN,NaN,Серёжа,4.801717e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60988,73764,message,2021-03-24T10:08:35,NaN,NaN,NaN,Спасибо большое! Попробую сегодня,NaN,NaN,ИриШок,5.225578e+09,NaN,NaN,NaN,NaN,NaN,NaN,73759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60989,73765,message,2021-03-24T10:08:35,NaN,NaN,NaN,"[{'type': 'mention_name', 'text': 'Серёжа', 'u...",NaN,NaN,ChatKeeperBot,4.848115e+09,NaN,NaN,NaN,NaN,NaN,NaN,73759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Пример предпоследнего сообщения

In [6]:
ham_raw1['text'][60988]

'Спасибо большое! Попробую сегодня'

Пример последнего сообщения от бота

In [7]:
ham_raw1['text'][60989]

[{'type': 'mention_name', 'text': 'Серёжа', 'user_id': 506749612},
 ', вашу репутацию увеличил ',
 {'type': 'mention_name', 'text': 'ИриШок', 'user_id': 930610950},
 '.\n Ваша репутация 8']

Список признаков

In [8]:
ham_raw1.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'text', 'title',
       'members', 'from', 'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'reply_to_message_id', 'photo',
       'edited', 'forwarded_from', 'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
       'contact_vcard', 'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 'poll.answers',
       'location_information.latitude', 'location_information.longitude'],
      dtype='object')

Уникальные значения по некоторым признакам

In [9]:
all_cols = ['id', 'type', 'date', 'actor', 'actor_id', 'action', 'text', 'title',
       'members', 'from', 'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'reply_to_message_id', 'photo',
       'edited', 'forwarded_from', 'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
       'contact_vcard', 'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 'poll.answers',
       'location_information.latitude', 'location_information.longitude']

some_cols = ['id', 'type', 'date', 'actor_id', 'action',  
         'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'reply_to_message_id', 'photo',
       'edited',  'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
        'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 
       'location_information.latitude', 'location_information.longitude']

for col in some_cols:
    print(col, ham_raw1[col].unique())

id [-999996617          1         99 ...      73759      73764      73765]
type ['service' 'message']
date ['2016-08-26T12:01:01' '2016-09-01T18:04:20' '2016-09-01T18:07:07' ...
 '2021-03-24T09:48:16' '2021-03-24T09:52:51' '2021-03-24T10:08:35']
actor_id [4.49716059e+09 9.66694934e+09 4.48584734e+09 ... 4.86776919e+09
 5.43563062e+09 5.47227313e+09]
action ['migrate_to_supergroup' 'migrate_from_group' 'invite_members' nan
 'remove_members' 'join_group_by_link' 'pin_message' 'group_call'
 'invite_to_group_call']
from_id [           nan 4.47956145e+09 4.55065251e+09 ... 5.37073416e+09
 5.95687004e+09 5.75695058e+09]
file [nan '(File not included. Change data exporting settings to download.)']
thumbnail [nan '(File not included. Change data exporting settings to download.)']
media_type [nan 'sticker' 'video_file' 'animation' 'voice_message' 'video_message'
 'audio_file']
sticker_emoji [nan '😠' '💩' '🎉' '🌚' '😊' '🙈' '😒' '🎮' '😂' '🙄' '👈' '😳' '🤔' '😌' '😞' '😋' '😍'
 '☺️' '😁' '👍' '✌' '💐' '😡' '🔥' '😐

In [10]:
ham_raw1['text'].tail(10)

60980    [{'type': 'mention_name', 'text': 'بلا أم أحمق...
60981    Здравствуйте! Скажите пожалуйста ,а машина сух...
60982           Здравствуйте, да, там сушка автоматическая
60983                       На объездном  сейчас очередь (
60984                              И не царапает покрытие?
60985                              Присоединяюсь к вопросу
60986    лично у меня ни разу не царапало.. но я езжу т...
60987    мойка проходит в несколько этапов: сначала сби...
60988                    Спасибо большое! Попробую сегодня
60989    [{'type': 'mention_name', 'text': 'Серёжа', 'u...
Name: text, dtype: object

In [11]:
# извлечение из json текста, , замена '' на Nan, удаление строк (сообщений) без текста
# обновление индекса после удаление строк (индекс теперь по порядку)
ham = ham_raw1['text'].apply(clean_msg).replace('', np.nan).dropna().reset_index(drop=True)
ham

0                                                  Всем ✋🏻
1        Это супергруппа. А это значит что админ может ...
2                                                        /
3                    не пугайтесь, немного почистил мусор)
4                                             Всем привет)
                               ...                        
54382                              И не царапает покрытие?
54383                              Присоединяюсь к вопросу
54384    лично у меня ни разу не царапало.. но я езжу т...
54385    мойка проходит в несколько этапов: сначала сби...
54386                    Спасибо большое! Попробую сегодня
Name: text, Length: 54387, dtype: object

In [12]:
# увеличение максимальной ширины столбца
pd.set_option('max_colwidth', 110)
ham.head(50)

0                                                                                                           Всем ✋🏻
1     Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...
2                                                                                                                 /
3                                                                             не пугайтесь, немного почистил мусор)
4                                                                                                      Всем привет)
5                                                                                                       Привет всем
6                                                                                                      Добрый вечер
7                                                                                                   Доброе утро! :)
8     Дети разные бывают - если это маленькие - то родителям обьяснил бы

## Обработка сообщений со спамом

In [13]:
spam_raw.head(3)

,name,type,id,messages
0,Журнал НИ,private_channel,9802778229,"{'id': 1, 'type': 'service', 'date': '2020-07-07T20:25:57', 'actor': 'Журнал НИ', 'actor_id': 9802778229, ..."
1,Журнал НИ,private_channel,9802778229,"{'id': 4, 'type': 'message', 'date': '2020-07-07T20:27:15', 'from': 'Журнал НИ', 'from_id': 9802778229, 't..."
2,Журнал НИ,private_channel,9802778229,"{'id': 5, 'type': 'message', 'date': '2020-07-07T20:31:15', 'from': 'Журнал НИ', 'from_id': 9802778229, 't..."


In [14]:
spam_raw2 = pd.json_normalize(spam_raw['messages'])
pd.set_option('display.max_columns', 50)
spam_raw2.tail(10)

,id,type,date,actor,actor_id,action,title,text,from,from_id
2003,2044,message,2021-03-23T13:22:14,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой mute\n1c565fd4-fff4-4d57-a7e1-e88444b10d3a\n\n, {'type': 'code', 'text': ...",Журнал НИ,9.802778e+09
2004,2045,message,2021-03-23T15:26:59,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой mute\n17aa7c06-b7bb-4ecc-a0f2-9beda0594b9c\n\n, {'type': 'code', 'text': ...",Журнал НИ,9.802778e+09
2005,2046,message,2021-03-23T21:30:20,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, , {'type': 'hashtag', 'text': '#новыйпользователь...",Журнал НИ,9.802778e+09
2006,2047,message,2021-03-23T21:34:33,NaN,NaN,NaN,NaN,"[⚙️ , {'type': 'hashtag', 'text': '#Настройки'}, Пользователь изменил настройки группы , {'type': 'hashta...",Журнал НИ,9.802778e+09
2007,2048,message,2021-03-24T09:12:42,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, , {'type': 'hashtag', 'text': '#новыйпользователь...",Журнал НИ,9.802778e+09
2008,2049,message,2021-03-24T09:13:01,NaN,NaN,NaN,NaN,"[✅ Пользователь прошедший проверку разблокирован , {'type': 'hashtag', 'text': '#запрос'}, \n133e204a-8a1b...",Журнал НИ,9.802778e+09
2009,2050,message,2021-03-24T09:14:22,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n6ae21a32-1bdf-47c5-a940-f2b13181e8aa\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09
2010,2051,message,2021-03-24T09:14:32,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n63942221-6053-4e57-af28-b0b54d7f7bc1\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09
2011,2052,message,2021-03-24T09:14:53,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n917036a2-8087-4607-94fa-60b632b866f7\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09
2012,2053,message,2021-03-24T09:18:28,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n4502aa45-f47b-4415-891b-5baf3368fde1\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09


In [15]:
spam_raw2.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'title', 'text',
       'from', 'from_id'],
      dtype='object')

In [16]:
for col in ['type', 'actor', 'actor_id', 'action', 'title', 'from', 'from_id']:
    print(col, spam_raw2[col].unique())

type ['service' 'message']
actor ['Журнал НИ' nan]
actor_id [9.80277823e+09            nan]
action ['create_channel' nan]
title ['Журнал НИ' nan]
from [nan 'Журнал НИ']
from_id [           nan 9.80277823e+09]


In [17]:
spam_raw3 = spam_raw2['text']
spam_raw3.tail(15)

1998    [🆘 Пользователь наказан командой mute\nd5bce06f-b34c-4994-8aea-0692697036d2\n\n, {'type': 'code', 'text': ...
1999    [🆘 Пользователь наказан командой mute\n3e7a9f94-d819-4e23-9174-f78cf59990e0\n\n, {'type': 'code', 'text': ...
2000    [🆘 Пользователь наказан командой mute\n577e6a18-48bb-4c13-9c89-a7c240574859\n\n, {'type': 'code', 'text': ...
2001    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'},  , {'type': 'hashtag', 'text': '#новыйпользователь...
2002    [✅ Пользователь прошедший проверку разблокирован , {'type': 'hashtag', 'text': '#запрос'}, \n09698fee-2773...
2003    [🆘 Пользователь наказан командой mute\n1c565fd4-fff4-4d57-a7e1-e88444b10d3a\n\n, {'type': 'code', 'text': ...
2004    [🆘 Пользователь наказан командой mute\n17aa7c06-b7bb-4ecc-a0f2-9beda0594b9c\n\n, {'type': 'code', 'text': ...
2005    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'},  , {'type': 'hashtag', 'text': '#новыйпользователь...
2006    [⚙️ , {'type': 'hashtag', 'text': '#Настройки'},

In [18]:
spam_raw3.iloc[2003]

['🆘 Пользователь наказан командой mute\n1c565fd4-fff4-4d57-a7e1-e88444b10d3a\n\n',
 {'type': 'code', 'text': 'Sunlighter'},
 ' [',
 {'type': 'mention', 'text': '@sunlighter'},
 '] [',
 {'type': 'hashtag', 'text': '#user1177305831'},
 '] ',
 {'type': 'italic', 'text': 'ограничил возможность писать сообщения'},
 ' ',
 {'type': 'code', 'text': 'МГТС'},
 ' [',
 {'type': 'mention_name', 'text': '1176395792', 'user_id': 1176395792},
 '] [',
 {'type': 'hashtag', 'text': '#user1176395792'},
 '] на 24 часов\n\n\n\n',
 {'type': 'bold', 'text': 'Группа:'},
 ' ',
 {'type': 'code', 'text': 'Новое Измайлово'},
 ' [',
 {'type': 'mention', 'text': '@new_izmailovo'},
 '] [',
 {'type': 'hashtag', 'text': '#chat1001077014744'},
 ']\n',
 {'type': 'bold', 'text': 'Инициатор:'},
 ' ',
 {'type': 'code', 'text': 'Sunlighter'},
 ' [',
 {'type': 'mention', 'text': '@sunlighter'},
 '] [',
 {'type': 'hashtag', 'text': '#user1177305831'},
 ']\n',
 {'type': 'bold', 'text': 'Пользователь:'},
 ' ',
 {'type': 'code', 

In [19]:
clean_msg(spam_raw3.iloc[2003])

'Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон: +7(499)2380071'

In [20]:
spam = spam_raw3.apply(clean_msg)
spam.tail(10)

2003    Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...
2004    Здравствуйте! Услуги Психолога!\n\nПсихолог с высшим образованием и практикой более 10 лет предлагаю услуг...
2005                                                                                                             None
2006                                                                                                             None
2007                                                                                                             None
2008                                                                                                             None
2009                                                                                                             None
2010                                                                                                             None
2011                                                    

In [21]:
# извлечение из json текста, удаление строк (сообщений) без текста,
# обновление индекса после удаление строк (индекс теперь по порядку)
spam = spam_raw3.apply(clean_msg).dropna().reset_index(drop=True)

In [22]:
spam

0                                                                                                                   
1                                                                             Канал успешно добавлен в журнал группы
2         Добрый вечер!Предлагаю услуги няни.Свободно до 1 сентября.без вредных привычек.Опыт большой  ,люблю детей.
3      Приветствую 🙂\nКУПЛЮ ОДНОКОМНАТНУЮ (от 38м) квартиру в Новом Измайлово\nс ремонтом, мебелью, бытовой техни...
4                                                  Черничка\nДоставка черники по Балашихе\nhttps://t.me/chernichka03
                                                           ...                                                      
182    Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...
183                                                                      https://t.me/MoscowskyPravdorub/1470?single
184                                                             

## Получение объединённого датасета

In [23]:
# Series -> DataFrame
ham_df = ham.to_frame('msg')
# создание нового столбца с категорией спам / не спам
ham_df['spam'] = 0

spam_df = spam.to_frame('msg')
spam_df['spam'] = 1

# конкатенация датафреймов
df = pd.concat([ham_df, spam_df], axis=0, ignore_index=True)
df

,msg,spam
0,Всем ✋🏻,0
1,Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...,0
2,/,0
3,"не пугайтесь, немного почистил мусор)",0
4,Всем привет),0
...,...,...
54569,"Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...",1
54570,https://t.me/MoscowskyPravdorub/1470?single,1
54571,Большой выбор ждём вас,1
54572,Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...,1


In [34]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('spam', 1), 
                                                    df['spam'], random_state=0)

In [40]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [42]:
with open('../../data/ml_in_business/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [43]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [44]:
%%time
#Запускаем очистку текста. Будет долго...
df['msg'] = df['msg'].apply(lambda x: clean_text(x), 1)

<ipython-input-43-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 1.48 s


In [45]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
df['msg'] = df['msg'].apply(lambda x: lemmatization(x), 1)

Wall time: 18.6 s


А теперь в 3 строчки обучим нашу модель

In [46]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in df['msg'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in df['msg'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [48]:
common_dictionary[11]

'обещать'

In [49]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 18.1 s


In [50]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [52]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in df['msg'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

[]


[(0, 0.04),
 (1, 0.04),
 (2, 0.04),
 (3, 0.04),
 (4, 0.04),
 (5, 0.04),
 (6, 0.04),
 (7, 0.04),
 (8, 0.04),
 (9, 0.04),
 (10, 0.04),
 (11, 0.04),
 (12, 0.04),
 (13, 0.04),
 (14, 0.04),
 (15, 0.04),
 (16, 0.04),
 (17, 0.04),
 (18, 0.04),
 (19, 0.04),
 (20, 0.04),
 (21, 0.04),
 (22, 0.04),
 (23, 0.04),
 (24, 0.04)]

In [53]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: машина работать свой строитель вообще тц ставить
topic_1: спасибо нужно нужный дать понять либо большой
topic_2: ребёнок проблема всё помочь двор хозяин счёт
topic_3: говорить утро сегодня неделя водитель смочь правило
topic_4: район наш это балашиха именно информация каждый
topic_5: дом это всё ситуация оплата факт остальной
topic_6: время который мочь рубль второй чтоть иметь
topic_7: весь год видеть хотеть привет платить ждать
topic_8: ещё парковка ситниковый дорога стоить поставить её
topic_9: место найти большой ходить ездить пусть любой
topic_10: квартира собака номер контакт около карта взять
topic_11: чат очень написать личка ремонт дело звонить
topic_12: писать первый интересно туда ехать сидеть несколько
topic_13: делать понимать хотя брать сразу предлагать продавать
topic_14: знать день площадка детский подъезд искать вода
topic_15: это сказать вроде деньга давать смотреть человек
topic_16: москва жить маленький работа кстати лс московский
topic_17: добрый сосед ден

In [54]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [57]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in df['msg'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['msg_id'] = df['msg_id'].values
topic_matrix = topic_matrix[['msg_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

KeyError: 'msg_id'